In [ ]:
import helper_functions

In [ ]:
# TODO - make it use helper functions
def Rfe(algorithm, num_inputing_algorithm= DecisionTreeRegressor() , cat_inputing_algorithm = DecisionTreeClassifier()):

    X_train, X_val,y_train, y_val = train_test_split(X,y,
                                                train_size = 0.75, 
                                                shuffle = True, 
                                                stratify = y)

    #Filling num missing values
    for column in num_features:
        impute_missing_values(X_train, column, num_inputing_algorithm)
        impute_missing_values(X_val, column, num_inputing_algorithm)

    #Filling cat missing values
    impute_missing_values(X_train, "Alternative Dispute Resolution", cat_inputing_algorithm)
    impute_missing_values(X_val, "Alternative Dispute Resolution", cat_inputing_algorithm)

    # Removing inconsistencies on the train
    inconsistent = X_train[(X_train['Age at Injury'] > 80) | (X_train["Age at Injury"] < 16)].index
    X_train.drop(inconsistent, inplace=True)
    y_train.drop(inconsistent, inplace=True)
    
    #Generating the variables where we will store our results
    nof_list = np.arange(1, len(X_train.columns) + 1)            
    high_score = 0
    opt_n_features = 0
    train_score_list = []
    val_score_list = []

    #Variable where we will store the optimum amount of features
    best_rfe = None

    model = algorithm

    for n in nof_list:
        rfe = RFE(estimator=model, n_features_to_select=n)
    
    # Fitting the model to rfe
        X_train_rfe = rfe.fit_transform(X_train, y_train)
        X_val_rfe = rfe.transform(X_val)
    
    # Training and predicting
        model.fit(X_train_rfe, y_train)
        pred_train = model.predict(X_train_rfe)
        pred_val = model.predict(X_val_rfe)
    
    # Evaluating using the macro f1_score
        train_score = f1_score(y_train, pred_train, average="macro")
        val_score = f1_score(y_val, pred_val, average="macro")
        train_score_list.append(train_score)
        val_score_list.append(val_score)
    
    # Checking if this is the best combination of features so far
        if val_score >= high_score:
            high_score = val_score
            opt_n_features = n
            best_rfe = rfe  # Armazenar o RFE com o melhor número de features

# Checking what amount of features and which features where the best for the model
    selected_features = X_train.columns[best_rfe.support_].tolist()

    print("Optimal number of features: %d" % opt_n_features)
    print("Score with %d features: %f" % (opt_n_features, high_score))
    print("Selected Features:\n", selected_features)
